In [1]:
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')
import importlib

from __init__ import *
from __init__jupyterlab import *
import snmcseq_utils
importlib.reload(snmcseq_utils)
import CEMBA_preproc_utils
importlib.reload(CEMBA_preproc_utils)
import CEMBA_clst_utils
import re

from scipy import sparse
import time

# from scipy.io import mmread
# import fbpca

In [2]:
import matplotlib
matplotlib.__version__

'3.1.3'

# Task

- start from prepared files 

```metadata``` ```count matrix```
- get and store hvfeatures

# Settings

In [3]:
# !cp ./data/organized_scf/__init__datasets.py ./data/test_Jan6_Astro/

In [4]:
SRC_DIR = './data/test_Jan6_Astro'
DST_DIR = './data/test_Jan6_Astro'
sys.path.insert(0, SRC_DIR)
from __init__datasets import *

f_meta_format = '{0}/{1}_metadata.tsv'
f_data_format = '{0}/{1}_{2}raw.{3}'
f_hvftr_format = '{0}/{1}_hvfeatures.{2}'
f_ftr_format = '{0}/{1}_features.{2}'

def get_size_in_GB(obj):
    """"""
    GB = 1024**3
    return sys.getsizeof(obj)/GB

In [5]:
mods_selected = [
    'snrna',
    'atac',
]

normalization_options = {
    'snrna': 'CPM',
    'atac': 'TPM',
}

In [6]:
# # # # gene name as index
# gene_annot_file = '/cndd2/fangming/projects/m3c/refs/gencode_v32_gene_annotation.bed' 
# gene_annot = pd.read_csv(gene_annot_file, sep="\t", names=['chr', 'start', 'end', 'gene_id', 'gene_name'])
# gene_annot_v2 = gene_annot.groupby('gene_name').first()
# print(gene_annot_v2.shape)
# gene_lengths_base = (gene_annot_v2['end'] - gene_annot_v2['start'])
# print(gene_lengths_base.head())
# get gene_lengths

def clean_gene_ids(x):
    """
    """
    start, end = re.search(".[0-9]+_", x).span()
    return x[:start]+"_"+x[end:]

f = os.path.join('./data/orig_Jan6', 'hg38_gencode_v28_genes.bed')
df_genes = pd.read_csv(f, sep="\t", header=None, 
                       names=['chr', 'start', 'end', 'id', 'score', 'strand']
                      )
df_genes['id'] = df_genes['id'].apply(clean_gene_ids)
df_genes = df_genes.set_index('id')
print(df_genes.shape)

gene_lengths_base = (df_genes['end'] - df_genes['start'])
print(gene_lengths_base.shape)
print(gene_lengths_base.head())

(58381, 5)
(58381,)
id
ENSG00000223972_DDX11L1          2541
ENSG00000227232_WASH7P          15167
ENSG00000278267_MIR6859-1          68
ENSG00000243485_RP11-34P13.3     1556
ENSG00000284332_MIR1302-2         138
dtype: int64


# highly variable features

In [7]:
for mod in mods_selected:
    ti = time.time()
    print(mod)
    
    # read metadata
    normalization_option = normalization_options[mod]
    f_meta = f_meta_format.format(SRC_DIR, mod) ##
    meta = pd.read_csv(f_meta, sep="\t", index_col=0)
    
    # read data matrix
    if normalization_option == 'MC':
        f_data_gene = f_data_format.format(SRC_DIR, mod, '', 'gene')
        f_data_cell = f_data_format.format(SRC_DIR, mod, '', 'cell')
        f_data_c = f_data_format.format(SRC_DIR, mod, 'CH_', 'npz')
        f_data_mc = f_data_format.format(SRC_DIR, mod, 'mCH_', 'npz')
        
        # read in files
        print(mod, "Reading in files {}".format(time.time()-ti))
        gxc_raw = snmcseq_utils.load_gc_matrix_methylation(f_data_gene, f_data_cell, f_data_mc, f_data_c)
        print(gxc_raw.data['mc'].shape, gxc_raw.data['c'].shape)
        print(time.time()-ti)
        
        # output file
        f_hvftr_data_methylation = f_hvftr_format.format(DST_DIR, mod, 'tsv') 
        print(time.time()-ti)
        
        # check meta cells agree with gxc cells
        assert np.all(meta.index.values == gxc_raw.cell)
        # check genes are uniq 
        assert len(gxc_raw.gene) == len(np.unique(gxc_raw.gene)) 
        
        # do
        gxc_hvftr = CEMBA_preproc_utils.preproc_methylation(
                                                            gxc_raw,
                                                            meta,
                                                            global_value_col=settings[mod].global_mean, 
                                                            base_call_cutoff=20, 
                                                            sufficient_coverage_fraction=0.95,
                                                            hv_percentile=30,
                                                            n_qcut=10,
                                                            )
        # save
        print(mod, "Saving to files {}".format(time.time()-ti))
        gxc_hvftr.to_csv(f_hvftr_data_methylation, sep="\t", header=True, index=True, na_rep='NA')
        
        
        
    else:
        f_data = f_data_format.format(SRC_DIR, mod, '', 'npz') 
        f_data_gene = f_data_format.format(SRC_DIR, mod, '', 'gene') 
        f_data_cell = f_data_format.format(SRC_DIR, mod, '', 'cell') 
        
        # read in files
        print(mod, "Reading in files {}".format(time.time()-ti))
        gxc_raw = snmcseq_utils.load_gc_matrix(f_data_gene, f_data_cell, f_data) # checked dimensions in agreement internally
        print(gxc_raw.data.shape)
        
        # output files
        f_hvftr_data = f_hvftr_format.format(DST_DIR, mod, 'npz') 
        f_hvftr_gene = f_hvftr_format.format(DST_DIR, mod, 'gene') 
        f_hvftr_cell = f_hvftr_format.format(DST_DIR, mod, 'cell') 
        
        # check meta cells agree with gxc cells
        assert np.all(meta.index.values == gxc_raw.cell)
        # check genes are uniq 
        assert len(gxc_raw.gene) == len(np.unique(gxc_raw.gene)) 
    
        # get hvftrs
        print(mod, "Preproc and get highly variable genes {}".format(time.time()-ti))
        if normalization_option == 'CPM':
            gxc_hvftr = CEMBA_preproc_utils.preproc_rna_cpm_based(
                                             gxc_raw, 
                                             sufficient_cell_coverage=0.01, 
                                             hv_percentile=30, hv_ncut=10)
            # save
            print(mod, "Saving to files {}".format(time.time()-ti))
            snmcseq_utils.save_gc_matrix(gxc_hvftr, f_hvftr_gene, f_hvftr_cell, f_hvftr_data)

        elif normalization_option == 'TPM':
            gene_lengths = gene_lengths_base.reindex(gxc_raw.gene)
            gxc_hvftr = CEMBA_preproc_utils.preproc_rna_tpm_based(
                                             gxc_raw, gene_lengths, impute_gene_lengths=True, 
                                             sufficient_cell_coverage=0.01, 
                                             hv_percentile=30, hv_ncut=10)
            # save
            print(mod, "Saving to files {}".format(time.time()-ti))
            snmcseq_utils.save_gc_matrix(gxc_hvftr, f_hvftr_gene, f_hvftr_cell, f_hvftr_data)
    
    
    print(mod, "Total time used: {}".format(time.time()-ti))
    

snrna
snrna Reading in files 0.07193613052368164
(28422, 13135)
snrna Preproc and get highly variable genes 1.9383690357208252
Removing low coverage genes...
Getting CPM..
Getting highly variable genes and logCPM...
Number of genes: 4365
snrna Saving to files 10.072838544845581
snrna Total time used: 14.597919940948486
atac
atac Reading in files 0.06397461891174316
(58381, 6385)
atac Preproc and get highly variable genes 1.8791122436523438
Imputing gene lengths...
Removing low coverage genes...
Getting CPM..
Getting highly variable genes and logCPM...
Getting logTPM...
Trim logTPM matrix...
Number of genes: 10449
atac Saving to files 16.506623029708862
atac Total time used: 21.08558678627014


## Check highly-variable genes

In [8]:
for mod in mods_selected:
    print(mod)
    if settings[mod].mod_category == 'mc':
        f_hvftr_data = f_hvftr_format.format(SRC_DIR, mod, 'tsv') 
        gxc_hvftr = pd.read_csv(f_hvftr_data, sep="\t", index_col=0)
        print(gxc_hvftr.index.values)
        print(gxc_hvftr.columns.values)
        print(gxc_hvftr.shape)
        has_nan = np.isnan(gxc_hvftr.values).any()
        print("Contains NaN? {}".format(has_nan))
        
        continue
        
    f_hvftr_data = f_hvftr_format.format(SRC_DIR, mod, 'npz') 
    f_hvftr_gene = f_hvftr_format.format(SRC_DIR, mod, 'gene') 
    f_hvftr_cell = f_hvftr_format.format(SRC_DIR, mod, 'cell') 
    gxc_hvftr = snmcseq_utils.load_gc_matrix(f_hvftr_gene, f_hvftr_cell, f_hvftr_data)
    print(gxc_hvftr.gene)
    print(gxc_hvftr.cell)
    print(len(gxc_hvftr.gene), len(gxc_hvftr.cell), gxc_hvftr.data.shape)
    has_nan = np.isnan(gxc_hvftr.data.data).any()
    print("Contains NaN? {}".format(has_nan))
#     break

snrna
['ENSG00000169972_PUSL1' 'ENSG00000272449_AL139246.5'
 'ENSG00000234546_LINC01759' ... 'ENSG00000160209_PDXK'
 'ENSG00000160307_S100B' 'ENSG00000198786_MT-ND5']
['MCX_ALS_332_AAGCGTTCACCTAAAC' 'MCX_ALS_332_AATGAAGAGTCCGCCA'
 'MCX_ALS_332_AATTCCTTCCGTTGAA' ... 'mFCX_Control_906_TTTGATCAGTCACACT'
 'mFCX_Control_906_TTTGGAGCACCGCTAG' 'mFCX_Control_906_TTTGTTGAGAGTGTTA']
4365 13135 (4365, 13135)
Contains NaN? False
atac
['ENSG00000216710_COX6A1P3' 'ENSG00000170688_OR5E1P'
 'ENSG00000230576_OR6R1P' ... 'ENSG00000231557_AC018717.1'
 'ENSG00000225885_AC023590.1' 'ENSG00000086758_HUWE1']
['MCX_ALS_110_AAACGAAAGAGATTAC' 'MCX_ALS_110_AACGAGGGTCGAGGAT'
 'MCX_ALS_110_ACATGCACAGGTTATC' ... 'mFCX_FTD_908_TTGCAGACAACTCAAC'
 'mFCX_FTD_908_TTGGTCCTCTCTGCAC' 'mFCX_FTD_908_TTTGGTTCATGCGTTA']
10449 6385 (10449, 6385)
Contains NaN? False
